# K-Nearest Neighbour 

In [1]:
import pandas as pd 
import numpy as np 

In [20]:
r_cols = ['user_id','movie_id','rating' ] #columns defined 
ratings = pd.read_csv('C:/Users/Ujjwal/Documents/Data_science/movierecommender/u.data',sep = '\t',names= r_cols,usecols=range(3)) #reading the csv data which are seperated by tab and the range is till 3
ratings.head()

,user_id,movie_id,rating
0,0,50,5
1,0,172,5
2,0,133,1
3,196,242,3
4,186,302,3


In [22]:
movierating = ratings.groupby('movie_id').agg({'rating':[np.size,np.mean]}) #group the movies by moviesID the take the sum of size and mean to value rating
                                                                #agg({}) is dictionary 
movierating.head()

rating          
           size      mean
movie_id                 
1           452  3.878319
2           131  3.206107
3            90  3.033333
4           209  3.550239
5            86  3.302326

In [25]:
movienumrating = pd.DataFrame(movierating['rating']['size'])
movienumrating.head()

,size
movie_id,
1,452
2,131
3,90
4,209
5,86


In [27]:
movienumrating = pd.DataFrame(movierating['rating']['size'])
movienormalizedrating = movienumrating.apply(lambda x:(x-np.min(x))/(np.max(x)-np.min(x))) #normal data cannot be inserted directly so we normalize 
movienormalizedrating.head()

,size
movie_id,
1,0.773585
2,0.222985
3,0.152659
4,0.356775
5,0.145798


In [49]:
movieDict = {}
with open(r'C:/Users/Ujjwal/Documents/Data_science/movierecommender/u.item',encoding = 'ISO-8859-1') as f :
    temp = ''
    
    for line in f:
        field = line.rstrip('\n').split('|')
        movieID = int(field[0])
        movieName = field[1]
        genres = field[5:25]
        genres = map(int,genres)
        movieDict[movieID] = (movieName,
                             np.asarray(list(genres)),
                             movienormalizedrating.loc[movieID].get('size'),
                             movierating.loc[movieID].rating.get('mean'))

In [53]:
movieDict[1]

('Toy Story (1995)',
 array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 0.7735849056603774,
 3.8783185840707963)

In [56]:
from scipy.spatial.distance import cosine #to compute the distance importing cosine 

In [58]:
#cosine distance evaluation practice 
a = np.asarray([1,0,1])
b = np.asarray([1,0,0])

In [60]:
cosine(a,b)

0.29289321881345254

In [ ]:
#use the cosine formula 

In [66]:
def computedistance(a,b):
    popularityA = a[2]
    popularityB = b[2]
    popularitydistance = abs(popularityA - popularityB) #
    genreA = a[1]
    genreB = b[1]
    genredistance = cosine(genreA, genreB)
    return popularitydistance + genredistance

In [72]:
print(movieDict[1])
print(movieDict[2])

#or 

computedistance(movieDict[1],movieDict[2])

('Toy Story (1995)', array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 0.7735849056603774, 3.8783185840707963)
('GoldenEye (1995)', array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]), 0.22298456260720412, 3.2061068702290076)


1.5506003430531732

In [91]:
import operator
def FindNearestMovie(M):
    distance = []
    for movie in movieDict:
        if movieDict[movie] != M:
            distance.append((movie,computedistance(M,movieDict[movie])))
    distance.sort(key=operator.itemgetter(1))
    return distance     

In [96]:
def KNearestMovie(M,k):
    nearestmovie = []
    movie = FindNearestMovie(M)
    for i in range(k):
        m = movie[i]
        a = movieDict[m[0]]
        nearestmovie.append(a[0])
    return nearestmovie

In [98]:
KNearestMovie(movieDict[1],10)

['Liar Liar (1997)',
 'Aladdin (1992)',
 'Willy Wonka and the Chocolate Factory (1971)',
 'Monty Python and the Holy Grail (1974)',
 'Full Monty, The (1997)',
 'George of the Jungle (1997)',
 'Beavis and Butt-head Do America (1996)',
 'Birdcage, The (1996)',
 'Home Alone (1990)',
 'Aladdin and the King of Thieves (1996)']